In [525]:
import sys
import cv2
import datetime
import os
import numpy as np
import os.path
import glob
import sklearn
import pandas as pd
import mahotas as mt
import imageio
import graphviz
from time import time
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import scale

In [526]:
BAN_LIST = {'stone2', 'oatmeal1', 'rug1', 'stone3'}

In [574]:
def extract_features(image, n_neighbours):
    # Вычисляем признаки Харалика для первого соседа
    ht_features = mt.features.haralick(image)
    ht_mean = ht_features.mean(axis=0)

    for i in range(2, n_neighbours + 1):
        # Вычисляем признаки харалика для i-го соседа
        ht_temp_features = mt.features.haralick(image, distance=i)

        # Складываем признаки в массив
        ht_mean = np.append(ht_mean, ht_temp_features.mean(axis=0))
    return ht_mean

In [578]:
def form_training_df(training_dir, n_neighbours, n_images):
    x_train = []
    y_train = []
    filenames = []
    for class_dir_name in os.listdir(training_dir):
        if class_dir_name in BAN_LIST:
            continue
        class_dir = training_dir + class_dir_name
        n_segments = len(os.listdir(class_dir))
        segments_step = n_segments // n_images - 1
        objects_count = 0
        for i in range(0, n_segments, segments_step):
            image_file = os.listdir(class_dir)[i]
            image = cv2.imread(class_dir + '\\' + image_file)

            # Перевод в серый и извлечение признаков Харалика
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            features = extract_features(gray, n_neighbours)

            # Выводит имя файла, попавшего в выборку (можно закомментировать)
            print(image_file)

            # Добавляем в массив признаки, метку класса и название файла
            x_train.append(features)
            y_train.append(class_dir_name)
            filenames.append(image_file)

            objects_count += 1
            if objects_count == n_images:
                break

    # Переводим массивы в структуру DataFrame из pandas
    x_train_df = pd.DataFrame(x_train, columns=[str(i) for i in range(1, n_neighbours * 13 + 1)])
    y_train_df = pd.DataFrame(y_train, columns=['Class'])
    filenames_df = pd.DataFrame(filenames, columns=['Filename'])

    return x_train_df, y_train_df, filenames_df

In [579]:
X_train, Y_train, Train_filenames = form_training_df('D:\\projects\\cutted_texture_dataset1_128\\', 20, 200)

blanket1-a-p001_[0_0].bmp
blanket1-a-p001_[260_390].bmp
blanket1-a-p002_[130_260].bmp
blanket1-a-p003_[0_130].bmp
blanket1-a-p003_[390_0].bmp
blanket1-a-p004_[130_390].bmp
blanket1-a-p005_[0_260].bmp
blanket1-a-p005_[390_130].bmp
blanket1-a-p006_[260_0].bmp
blanket1-a-p007_[0_390].bmp
blanket1-a-p007_[390_260].bmp
blanket1-a-p008_[260_130].bmp
blanket1-a-p009_[130_0].bmp
blanket1-a-p009_[390_390].bmp
blanket1-a-p010_[260_260].bmp
blanket1-a-p011_[130_130].bmp
blanket1-a-p012_[0_0].bmp
blanket1-a-p012_[260_390].bmp
blanket1-a-p013_[130_260].bmp
blanket1-a-p014_[0_130].bmp
blanket1-a-p014_[390_0].bmp
blanket1-a-p015_[130_390].bmp
blanket1-a-p016_[0_260].bmp
blanket1-a-p016_[390_130].bmp
blanket1-a-p017_[260_0].bmp
blanket1-a-p018_[0_390].bmp
blanket1-a-p018_[390_260].bmp
blanket1-a-p019_[260_130].bmp
blanket1-a-p020_[130_0].bmp
blanket1-a-p020_[390_390].bmp
blanket1-a-p021_[260_260].bmp
blanket1-a-p022_[130_130].bmp
blanket1-a-p023_[0_0].bmp
blanket1-a-p023_[260_390].bmp
blanket1-a-p024_

blanket2-b-p018_[0_130].bmp
blanket2-b-p018_[390_0].bmp
blanket2-b-p019_[130_390].bmp
blanket2-b-p020_[0_260].bmp
blanket2-b-p020_[390_130].bmp
blanket2-b-p021_[260_0].bmp
blanket2-b-p022_[0_390].bmp
blanket2-b-p022_[390_260].bmp
blanket2-b-p023_[260_130].bmp
blanket2-b-p024_[130_0].bmp
blanket2-b-p024_[390_390].bmp
blanket2-b-p025_[260_260].bmp
blanket2-b-p026_[130_130].bmp
blanket2-b-p027_[0_0].bmp
blanket2-b-p027_[260_390].bmp
blanket2-b-p028_[130_260].bmp
blanket2-b-p029_[0_130].bmp
blanket2-b-p029_[390_0].bmp
blanket2-b-p030_[130_390].bmp
blanket2-b-p031_[0_260].bmp
blanket2-b-p031_[390_130].bmp
blanket2-b-p032_[260_0].bmp
blanket2-b-p033_[0_390].bmp
blanket2-b-p033_[390_260].bmp
blanket2-b-p034_[260_130].bmp
blanket2-b-p035_[130_0].bmp
blanket2-b-p035_[390_390].bmp
blanket2-b-p036_[260_260].bmp
blanket2-b-p037_[130_130].bmp
blanket2-b-p038_[0_0].bmp
blanket2-b-p038_[260_390].bmp
blanket2-b-p039_[130_260].bmp
blanket2-b-p040_[0_130].bmp
blanket2-b-p040_[390_0].bmp
blanket2-c-p001_

canvas1-c-p039_[130_0].bmp
canvas1-c-p039_[390_390].bmp
canvas1-c-p040_[260_260].bmp
canvas1-d-p001_[130_130].bmp
canvas1-d-p002_[0_0].bmp
canvas1-d-p002_[260_390].bmp
canvas1-d-p003_[130_260].bmp
canvas1-d-p004_[0_130].bmp
canvas1-d-p004_[390_0].bmp
canvas1-d-p005_[130_390].bmp
canvas1-d-p006_[0_260].bmp
canvas1-d-p006_[390_130].bmp
canvas1-d-p007_[260_0].bmp
canvas1-d-p008_[0_390].bmp
canvas1-d-p008_[390_260].bmp
canvas1-d-p009_[260_130].bmp
canvas1-d-p010_[130_0].bmp
canvas1-d-p010_[390_390].bmp
canvas1-d-p011_[260_260].bmp
canvas1-d-p012_[130_130].bmp
canvas1-d-p013_[0_0].bmp
canvas1-d-p013_[260_390].bmp
canvas1-d-p014_[130_260].bmp
canvas1-d-p015_[0_130].bmp
canvas1-d-p015_[390_0].bmp
canvas1-d-p016_[130_390].bmp
canvas1-d-p017_[0_260].bmp
canvas1-d-p017_[390_130].bmp
ceiling1-a-p001_[0_0].bmp
ceiling1-a-p001_[260_390].bmp
ceiling1-a-p002_[130_260].bmp
ceiling1-a-p003_[0_130].bmp
ceiling1-a-p003_[390_0].bmp
ceiling1-a-p004_[130_390].bmp
ceiling1-a-p005_[0_260].bmp
ceiling1-a-p005_

ceiling2-a-p039_[260_0].bmp
ceiling2-a-p040_[0_390].bmp
ceiling2-a-p040_[390_260].bmp
ceiling2-b-p001_[260_130].bmp
ceiling2-b-p002_[130_0].bmp
ceiling2-b-p002_[390_390].bmp
ceiling2-b-p003_[260_260].bmp
ceiling2-b-p004_[130_130].bmp
ceiling2-b-p005_[0_0].bmp
ceiling2-b-p005_[260_390].bmp
ceiling2-b-p006_[130_260].bmp
ceiling2-b-p007_[0_130].bmp
ceiling2-b-p007_[390_0].bmp
ceiling2-b-p008_[130_390].bmp
ceiling2-b-p009_[0_260].bmp
ceiling2-b-p009_[390_130].bmp
ceiling2-b-p010_[260_0].bmp
ceiling2-b-p011_[0_390].bmp
ceiling2-b-p011_[390_260].bmp
ceiling2-b-p012_[260_130].bmp
ceiling2-b-p013_[130_0].bmp
ceiling2-b-p013_[390_390].bmp
ceiling2-b-p014_[260_260].bmp
ceiling2-b-p015_[130_130].bmp
ceiling2-b-p016_[0_0].bmp
ceiling2-b-p016_[260_390].bmp
ceiling2-b-p017_[130_260].bmp
ceiling2-b-p018_[0_130].bmp
ceiling2-b-p018_[390_0].bmp
ceiling2-b-p019_[130_390].bmp
ceiling2-b-p020_[0_260].bmp
ceiling2-b-p020_[390_130].bmp
ceiling2-b-p021_[260_0].bmp
ceiling2-b-p022_[0_390].bmp
ceiling2-b-p022_

cushion1-c-p016_[260_130].bmp
cushion1-c-p017_[130_0].bmp
cushion1-c-p017_[390_390].bmp
cushion1-c-p018_[260_260].bmp
cushion1-c-p019_[130_130].bmp
cushion1-c-p020_[0_0].bmp
cushion1-c-p020_[260_390].bmp
cushion1-c-p021_[130_260].bmp
cushion1-c-p022_[0_130].bmp
cushion1-c-p022_[390_0].bmp
cushion1-c-p023_[130_390].bmp
cushion1-c-p024_[0_260].bmp
cushion1-c-p024_[390_130].bmp
cushion1-c-p025_[260_0].bmp
cushion1-c-p026_[0_390].bmp
cushion1-c-p026_[390_260].bmp
cushion1-c-p027_[260_130].bmp
cushion1-c-p028_[130_0].bmp
cushion1-c-p028_[390_390].bmp
cushion1-c-p029_[260_260].bmp
cushion1-c-p030_[130_130].bmp
cushion1-c-p031_[0_0].bmp
cushion1-c-p031_[260_390].bmp
cushion1-c-p032_[130_260].bmp
cushion1-c-p033_[0_130].bmp
cushion1-c-p033_[390_0].bmp
cushion1-c-p034_[130_390].bmp
cushion1-c-p035_[0_260].bmp
cushion1-c-p035_[390_130].bmp
cushion1-c-p036_[260_0].bmp
cushion1-c-p037_[0_390].bmp
cushion1-c-p037_[390_260].bmp
cushion1-c-p038_[260_130].bmp
cushion1-c-p039_[130_0].bmp
cushion1-c-p03

...

screen1-b-p038_[0_0].bmp
screen1-b-p038_[260_390].bmp
screen1-b-p039_[130_260].bmp
screen1-b-p040_[0_130].bmp
screen1-b-p040_[390_0].bmp
screen1-c-p001_[130_390].bmp
screen1-c-p002_[0_260].bmp
screen1-c-p002_[390_130].bmp
screen1-c-p003_[260_0].bmp
screen1-c-p004_[0_390].bmp
screen1-c-p004_[390_260].bmp
screen1-c-p005_[260_130].bmp
screen1-c-p006_[130_0].bmp
screen1-c-p006_[390_390].bmp
screen1-c-p007_[260_260].bmp
screen1-c-p008_[130_130].bmp
screen1-c-p009_[0_0].bmp
screen1-c-p009_[260_390].bmp
screen1-c-p010_[130_260].bmp
screen1-c-p011_[0_130].bmp
screen1-c-p011_[390_0].bmp
screen1-c-p012_[130_390].bmp
screen1-c-p013_[0_260].bmp
screen1-c-p013_[390_130].bmp
screen1-c-p014_[260_0].bmp
screen1-c-p015_[0_390].bmp
screen1-c-p015_[390_260].bmp
screen1-c-p016_[260_130].bmp
screen1-c-p017_[130_0].bmp
screen1-c-p017_[390_390].bmp
screen1-c-p018_[260_260].bmp
screen1-c-p019_[130_130].bmp
screen1-c-p020_[0_0].bmp
screen1-c-p020_[260_390].bmp
screen1-c-p021_[130_260].bmp
screen1-c-p022_[0_130

seat2-a-p015_[130_390].bmp
seat2-a-p016_[0_260].bmp
seat2-a-p016_[390_130].bmp
seat2-a-p017_[260_0].bmp
seat2-a-p018_[0_390].bmp
seat2-a-p018_[390_260].bmp
seat2-a-p019_[260_130].bmp
seat2-a-p020_[130_0].bmp
seat2-a-p020_[390_390].bmp
seat2-a-p021_[260_260].bmp
seat2-a-p022_[130_130].bmp
seat2-a-p023_[0_0].bmp
seat2-a-p023_[260_390].bmp
seat2-a-p024_[130_260].bmp
seat2-a-p025_[0_130].bmp
seat2-a-p025_[390_0].bmp
seat2-a-p026_[130_390].bmp
seat2-a-p027_[0_260].bmp
seat2-a-p027_[390_130].bmp
seat2-a-p028_[260_0].bmp
seat2-a-p029_[0_390].bmp
seat2-a-p029_[390_260].bmp
seat2-a-p030_[260_130].bmp
seat2-a-p031_[130_0].bmp
seat2-a-p031_[390_390].bmp
seat2-a-p032_[260_260].bmp
seat2-a-p033_[130_130].bmp
seat2-a-p034_[0_0].bmp
seat2-a-p034_[260_390].bmp
seat2-a-p035_[130_260].bmp
seat2-a-p036_[0_130].bmp
seat2-a-p036_[390_0].bmp
seat2-a-p037_[130_390].bmp
seat2-a-p038_[0_260].bmp
seat2-a-p038_[390_130].bmp
seat2-a-p039_[260_0].bmp
seat2-a-p040_[0_390].bmp
seat2-a-p040_[390_260].bmp
seat2-b-p001

sesameseeds1-b-p035_[130_0].bmp
sesameseeds1-b-p035_[390_390].bmp
sesameseeds1-b-p036_[260_260].bmp
sesameseeds1-b-p037_[130_130].bmp
sesameseeds1-b-p038_[0_0].bmp
sesameseeds1-b-p038_[260_390].bmp
sesameseeds1-b-p039_[130_260].bmp
sesameseeds1-b-p040_[0_130].bmp
sesameseeds1-b-p040_[390_0].bmp
sesameseeds1-c-p001_[130_390].bmp
sesameseeds1-c-p002_[0_260].bmp
sesameseeds1-c-p002_[390_130].bmp
sesameseeds1-c-p003_[260_0].bmp
sesameseeds1-c-p004_[0_390].bmp
sesameseeds1-c-p004_[390_260].bmp
sesameseeds1-c-p005_[260_130].bmp
sesameseeds1-c-p006_[130_0].bmp
sesameseeds1-c-p006_[390_390].bmp
sesameseeds1-c-p007_[260_260].bmp
sesameseeds1-c-p008_[130_130].bmp
sesameseeds1-c-p009_[0_0].bmp
sesameseeds1-c-p009_[260_390].bmp
sesameseeds1-c-p010_[130_260].bmp
sesameseeds1-c-p011_[0_130].bmp
sesameseeds1-c-p011_[390_0].bmp
sesameseeds1-c-p012_[130_390].bmp
sesameseeds1-c-p013_[0_260].bmp
sesameseeds1-c-p013_[390_130].bmp
sesameseeds1-c-p014_[260_0].bmp
sesameseeds1-c-p015_[0_390].bmp
sesameseeds1

stone1-d-p013_[0_0].bmp
stone1-d-p013_[260_390].bmp
stone1-d-p014_[130_260].bmp
stone1-d-p015_[0_130].bmp
stone1-d-p015_[390_0].bmp
stone1-d-p016_[130_390].bmp
stone1-d-p017_[0_260].bmp
stone1-d-p017_[390_130].bmp
stoneslab1-a-p001_[0_0].bmp
stoneslab1-a-p001_[260_390].bmp
stoneslab1-a-p002_[130_260].bmp
stoneslab1-a-p003_[0_130].bmp
stoneslab1-a-p003_[390_0].bmp
stoneslab1-a-p004_[130_390].bmp
stoneslab1-a-p005_[0_260].bmp
stoneslab1-a-p005_[390_130].bmp
stoneslab1-a-p006_[260_0].bmp
stoneslab1-a-p007_[0_390].bmp
stoneslab1-a-p007_[390_260].bmp
stoneslab1-a-p008_[260_130].bmp
stoneslab1-a-p009_[130_0].bmp
stoneslab1-a-p009_[390_390].bmp
stoneslab1-a-p010_[260_260].bmp
stoneslab1-a-p011_[130_130].bmp
stoneslab1-a-p012_[0_0].bmp
stoneslab1-a-p012_[260_390].bmp
stoneslab1-a-p013_[130_260].bmp
stoneslab1-a-p014_[0_130].bmp
stoneslab1-a-p014_[390_0].bmp
stoneslab1-a-p015_[130_390].bmp
stoneslab1-a-p016_[0_260].bmp
stoneslab1-a-p016_[390_130].bmp
stoneslab1-a-p017_[260_0].bmp
stoneslab1-a-p

wall1-b-p008_[130_390].bmp
wall1-b-p009_[0_260].bmp
wall1-b-p009_[390_130].bmp
wall1-b-p010_[260_0].bmp
wall1-b-p011_[0_390].bmp
wall1-b-p011_[390_260].bmp
wall1-b-p012_[260_130].bmp
wall1-b-p013_[130_0].bmp
wall1-b-p013_[390_390].bmp
wall1-b-p014_[260_260].bmp
wall1-b-p015_[130_130].bmp
wall1-b-p016_[0_0].bmp
wall1-b-p016_[260_390].bmp
wall1-b-p017_[130_260].bmp
wall1-b-p018_[0_130].bmp
wall1-b-p018_[390_0].bmp
wall1-b-p019_[130_390].bmp
wall1-b-p020_[0_260].bmp
wall1-b-p020_[390_130].bmp
wall1-b-p021_[260_0].bmp
wall1-b-p022_[0_390].bmp
wall1-b-p022_[390_260].bmp
wall1-b-p023_[260_130].bmp
wall1-b-p024_[130_0].bmp
wall1-b-p024_[390_390].bmp
wall1-b-p025_[260_260].bmp
wall1-b-p026_[130_130].bmp
wall1-b-p027_[0_0].bmp
wall1-b-p027_[260_390].bmp
wall1-b-p028_[130_260].bmp
wall1-b-p029_[0_130].bmp
wall1-b-p029_[390_0].bmp
wall1-b-p030_[130_390].bmp
wall1-b-p031_[0_260].bmp
wall1-b-p031_[390_130].bmp
wall1-b-p032_[260_0].bmp
wall1-b-p033_[0_390].bmp
wall1-b-p033_[390_260].bmp
wall1-b-p034

In [580]:
X_train

,1,2,3,4,5,6,7,8,9,10,...,251,252,253,254,255,256,257,258,259,260
0,0.000338,546.217505,0.818068,1501.127186,0.062801,238.287072,5458.291240,8.050751,11.914670,0.000096,...,1462.178014,0.031232,237.445630,3699.698165,7.785805,12.459971,0.000040,6.463420,-0.065478,0.754217
1,0.000333,605.864183,0.814005,1628.941669,0.061472,263.733312,5909.902492,8.073128,11.979896,0.000091,...,1636.473550,0.029829,264.492005,4122.354857,7.848438,12.533275,0.000038,6.540932,-0.067557,0.763203
2,0.000291,615.581390,0.813654,1652.192852,0.060415,259.564209,5993.190019,8.133164,12.178334,0.000085,...,1610.313166,0.026561,256.708741,3618.176256,7.807964,12.696600,0.000031,6.701949,-0.070444,0.776473
3,0.000295,554.774232,0.801508,1397.125755,0.061818,237.943986,5033.728788,8.011837,12.146229,0.000089,...,1379.375913,0.031414,238.483047,3344.635517,7.768738,12.631577,0.000038,6.533554,-0.074087,0.787490
4,0.000296,536.031894,0.799619,1337.806536,0.062397,249.360951,4815.194251,8.006556,12.134490,0.000091,...,1324.219107,0.030071,247.802740,3116.279432,7.718151,12.622544,0.000036,6.540284,-0.073394,0.785187
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4795,0.000221,312.513586,0.890658,1429.391290,0.083315,259.355323,5405.051575,8.156527,12.531839,0.000117,...,1400.423059,0.024436,258.832763,2754.456662,7.710846,13.314890,0.000026,6.777738,-0.098658,0.864745
4796,0.000171,425.333326,0.880901,1785.826259,0.071740,260.252855,6717.971711,8.316868,12.844183,0.000098,...,1813.659650,0.021067,260.061872,3536.826249,7.889473,13.553079,0.000021,6.966458,-0.111447,0.892373
4797,0.000284,243.755118,0.925544,1636.872881,0.092839,255.590997,6303.736405,8.270108,12.158540,0.000133,...,1685.667527,0.022910,254.242077,3269.918882,7.830549,13.243775,0.000022,6.909426,-0.091595,0.847475
4798,0.000191,318.277572,0.896775,1542.214862,0.077463,255.871988,5850.581876,8.253446,12.681332,0.000117,...,1524.555183,0.022840,255.657939,3032.803217,7.794786,13.448680,0.000024,6.827027,-0.108356,0.885804


In [581]:
cls = DecisionTreeClassifier(random_state=241, min_samples_split=0.02)

In [582]:
fold_generator = KFold(n_splits=5, shuffle = True, random_state=42)  # Генератор разбивания 
scorings = cross_val_score(cls, X_train, Y_train, cv=fold_generator)  # точность на кросс-валидации
print(scorings.mean())

0.9356249999999999


In [583]:
cls.fit(X_train, Y_train)

DecisionTreeClassifier(min_samples_split=0.02, random_state=241)

In [584]:
Y_pred = cls.predict(X_train)  # Вектор предсказаний для обучающей выборки

In [585]:
confusion_matrix(Y_train, Y_pred)  # Матрица ошибок

array([[199,   0,   0,   1,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  3, 190,   0,   0,   0,   0,   2,   0,   0,   2,   0,   2,   0,
          0,   0,   0,   0,   0,   0,   0,   1,   0,   0,   0],
       [  2,   0, 198,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0, 199,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   1,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0, 200,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  2,   1,   0,   0,   0, 187,   0,   0,   8,   2,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0, 200,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0

In [586]:
pd.DataFrame(cls.predict_proba(X_train)*100, columns=cls.classes_)  # Предсказываем принадлежность к классу выборочных данных

,blanket1,blanket2,canvas1,ceiling1,ceiling2,cushion1,floor1,floor2,grass1,lentils1,...,sand1,scarf1,scarf2,screen1,seat1,seat2,sesameseeds1,stone1,stoneslab1,wall1
0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4795,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0
4796,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0
4797,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,35.0,0.0,0.0,0.0,0.0,0.0,5.0,10.0,0.0,45.0
4798,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0


In [587]:
cls.score(X_train, Y_train) # точность на обучающей выборке

0.9735416666666666

In [542]:
X_train.to_csv('X_train_128_260.csv')  # Запись данных в csv файл 

In [543]:
Y_train.to_csv('Y_train_128_260.csv')

In [546]:
# X_readed = pd.read_csv('X_train_128_260.csv', index_col=0) 

In [577]:
X_test, Y_test, Test_Filenames = form_training_df('D:\\projects\\texture_dataset2\\', 20, 3)

banded_0025.jpg
banded_0075.jpg
banded_0112.jpg
blotchy_0021.jpg
blotchy_0063.jpg
blotchy_0102.jpg
braided_0042.jpg
braided_0122.jpg
braided_0158.jpg
bubbly_0050.jpg
bubbly_0096.jpg
bubbly_0134.jpg
bumpy_0072.jpg
bumpy_0128.jpg
bumpy_0164.jpg
chequered_0017.jpg
chequered_0113.jpg
chequered_0174.jpg
cobwebbed_0030.jpg
cobwebbed_0081.jpg
cobwebbed_0120.jpg
cracked_0004.jpg
cracked_0085.jpg
cracked_0126.jpg
crosshatched_0006.jpg
crosshatched_0092.jpg
crosshatched_0131.jpg
crystalline_0057.jpg
crystalline_0140.jpg
crystalline_0179.jpg
dotted_0001.jpg
dotted_0120.jpg
dotted_0163.jpg
fibrous_0035.jpg
fibrous_0136.jpg
fibrous_0175.jpg
flecked_0003.jpg
flecked_0087.jpg
flecked_0128.jpg
freckled_0009.jpg
freckled_0098.jpg
freckled_0137.jpg
frilly_0001.jpg
frilly_0041.jpg
frilly_0080.jpg
gauzy_0008.jpg
gauzy_0123.jpg
gauzy_0162.jpg
grid_0001.jpg
grid_0051.jpg
grid_0093.jpg
grooved_0001.jpg
grooved_0089.jpg
grooved_0128.jpg
honeycombed_0003.jpg
honeycombed_0095.jpg
honeycombed_0134.jpg
set.jpg
__

In [560]:
pd.DataFrame(cls.predict_proba(X_test)*100, columns=cls.classes_)

,blanket1,blanket2,canvas1,ceiling1,ceiling2,cushion1,floor1,floor2,grass1,lentils1,...,sand1,scarf1,scarf2,screen1,seat1,seat2,sesameseeds1,stone1,stoneslab1,wall1
0,0.0,100.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,81.818182,0.0,...,0.0,0.000000,0.0,9.090909,0.0,0.0,0.0,9.090909,0.0,0.0
2,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
3,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
4,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,0.0,100.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
140,0.0,81.818182,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,18.181818,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
141,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,100.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
142,0.0,0.000000,0.0,100.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
